<a href="https://colab.research.google.com/github/detektor777/colab_list_video/blob/main/google.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**🔗 1**. Open Google Cloud Console
Go to: https://console.cloud.google.com/apis/credentials

-

**🔍 2.** Enable **Google Drive API**
In the search bar, type: Google Drive API
Click on it, then press "**Enable**"

-

**🛠️ 3**. Create Credentials
Click the + **Create Credentials** button

Select:

**Google Drive API**

**Application data**

Click **Next**

-

**📝 4**. Configure the Service Account
In the first field, enter a unique name (e.g., drive-automation)

Click **Create and Continue**

-

**👤 5**. Set **Role**
Choose:
**Role > Basic > Editor**

Click Continue, then Done

-

🔐 6. Manage Service Account Keys
Go to: https://console.cloud.google.com/iam-admin/serviceaccounts

Select your newly created service account from the list

-

**🧷 7**. Create a JSON Key
In the "Actions" column, click the three dots (**⋮**)

Select "**Manage keys**"

Click "**Add key**" → "**Create new key**"

Choose **JSON**, then click **Create**

The .json key file will download automatically

-

**🧾 8**. Rename the JSON File
Rename the downloaded file using the service account’s email (for easier reference)
For example:
test@gmail.com → test.json

In [ ]:
#@title ##**Load json keys** { display-mode: "form" }
from google.colab import files
import json
import os

# Создаем директорию для хранения JSON файлов
if not os.path.exists('service_accounts'):
    os.makedirs('service_accounts')

# Загрузка нескольких JSON файлов
uploaded = files.upload()

# Сохраняем загруженные файлы
json_files = []
for filename, content in uploaded.items():
    if filename.endswith('.json'):
        filepath = os.path.join('service_accounts', filename)
        with open(filepath, 'wb') as f:
            f.write(content)
        json_files.append(filename)
        print(f'Сохранен файл: {filename}')

print('\nЗагруженные JSON файлы:')
for idx, filename in enumerate(json_files, 1):
    print(f'{idx}. {filename}')

In [ ]:
#@title ##**Google Drives** { display-mode: "form" }

from googleapiclient.discovery import build
from google.oauth2.service_account import Credentials
import ipywidgets as widgets
from IPython.display import display, clear_output
import os
import logging
import time

logging.getLogger('googleapiclient.discovery_cache').setLevel(logging.ERROR)

# Load all .json files from the service_accounts folder
json_files = [f for f in os.listdir('service_accounts') if f.endswith('.json')]

# Dropdown to select the service account JSON file
json_selector = widgets.Dropdown(
    options=json_files,
    description='JSON File:',
    style={'description_width': 'initial'}
)

# Dropdown to choose an action
action_selector = widgets.Dropdown(
    options=[
        'List files and folders in root',
        'List all files and folders',
        'Check available storage space',
        'Delete all files and folders'
    ],
    description='Action:',
    style={'description_width': 'initial'}
)

# Button to start the selected action
button = widgets.Button(description='Start')
output = widgets.Output()

def execute_action(b):
    with output:
        clear_output()

        try:
            SCOPES = ['https://www.googleapis.com/auth/drive']
            credentials = Credentials.from_service_account_file(
                os.path.join('service_accounts', json_selector.value),
                scopes=SCOPES
            )
            service = build('drive', 'v3', credentials=credentials)

            if action_selector.value == 'List files and folders in root':
                results = service.files().list(
                    pageSize=100,
                    fields="files(id, name, mimeType, size, modifiedTime)",
                    q="'root' in parents and trashed=false"
                ).execute()

                items = results.get('files', [])
                if not items:
                    print('No files found.')
                else:
                    for item in items:
                        file_type = 'Folder' if item['mimeType'] == 'application/vnd.google-apps.folder' else 'File'
                        print(f"{file_type}: {item['name']} (ID: {item['id']})")
                        print('-' * 50)

            elif action_selector.value == 'List all files and folders':
                results = service.files().list(
                    pageSize=1000,
                    fields="files(id, name, mimeType, size, modifiedTime)",
                    q="trashed=false"
                ).execute()

                items = results.get('files', [])
                if not items:
                    print('No files found.')
                else:
                    for item in items:
                        file_type = 'Folder' if item['mimeType'] == 'application/vnd.google-apps.folder' else 'File'
                        print(f"{file_type}: {item['name']} (ID: {item['id']})")
                        print('-' * 50)

            elif action_selector.value == 'Check available storage space':
                drive_info = service.about().get(fields="storageQuota").execute()
                storage_info = drive_info.get('storageQuota', {})

                try:
                    limit = int(storage_info.get('limit', 0))
                    usage = int(storage_info.get('usage', 0))
                    available = limit - usage

                    print("Storage Information:")
                    print(f"Total: {limit / (1024 ** 3):.2f} GB")
                    print(f"Used: {usage / (1024 ** 3):.2f} GB")
                    print(f"Available: {available / (1024 ** 3):.2f} GB")
                except (ValueError, TypeError) as e:
                    print("Error processing storage info:", str(e))

            elif action_selector.value == 'Delete all files and folders':
                print("Starting to delete all files and folders...")
                results = service.files().list(
                    pageSize=1000,
                    fields="files(id)",
                    q="trashed=false"
                ).execute()

                items = results.get('files', [])
                total_files = len(items)
                if total_files == 0:
                    print("No files or folders to delete.")
                    return

                progress = widgets.FloatProgress(
                    value=0.0,
                    min=0.0,
                    max=total_files,
                    description='Deleting:',
                    bar_style=''  # Options: 'success', 'info', 'warning', 'danger'
                )
                display(progress)

                # Delete files one by one
                for i, item in enumerate(items):
                    try:
                        service.files().delete(fileId=item['id']).execute()
                        progress.value += 1
                    except Exception as e:
                        print(f"Error deleting file ID {item['id']}: {str(e)}")

                progress.bar_style = 'success'
                print('All files and folders were successfully deleted.')

        except Exception as e:
            print(f"An error occurred: {str(e)}")

button.on_click(execute_action)

# Display UI
display(json_selector)
display(action_selector)
display(button)
display(output)


In [ ]:
#@title ##**Google Drive Explorer** { display-mode: "form" }
from googleapiclient.discovery import build
from google.oauth2.service_account import Credentials
import ipywidgets as widgets
from IPython.display import display, clear_output
import os
import logging
import io
import zipfile
from googleapiclient.http import MediaIoBaseDownload, MediaFileUpload
import time
import uuid
from google.colab import files
import tempfile

logging.getLogger('googleapiclient.discovery_cache').setLevel(logging.ERROR)

json_files = [f for f in os.listdir('service_accounts') if f.endswith('.json')]

json_selector = widgets.Dropdown(
    options=json_files,
    description='JSON file:',
    style={'description_width': 'initial'}
)

refresh_button = widgets.Button(description='Refresh')
file_list = widgets.Select(
    options=[],
    description='Files:',
    rows=10,
    layout=widgets.Layout(width='50%')
)
go_button = widgets.Button(description='Enter')
up_button = widgets.Button(description='Up')
download_button = widgets.Button(description='Download')
delete_button = widgets.Button(description='Delete')
upload_button = widgets.Button(description='Upload')
path_label = widgets.Label(value='Current path: /')
progress_bar = widgets.FloatProgress(
    value=0.0,
    min=0.0,
    max=1.0,
    description='Progress:',
    bar_style='info',
    layout={'visibility': 'hidden'}
)
output = widgets.Output()

current_folder_id = 'root'
folder_stack = []
is_downloading = False
last_click_time = 0
current_download_id = None

def get_service():
    try:
        SCOPES = ['https://www.googleapis.com/auth/drive']
        credentials = Credentials.from_service_account_file(
            os.path.join('service_accounts', json_selector.value),
            scopes=SCOPES
        )
        return build('drive', 'v3', credentials=credentials)
    except Exception as e:
        with output:
            print(f"Error connecting to Google Drive: {str(e)}")
        raise

def list_files(folder_id):
    try:
        service = get_service()
        files = []
        page_token = None
        while True:
            results = service.files().list(
                pageSize=1000,
                fields="nextPageToken, files(id, name, mimeType)",
                q=f"'{folder_id}' in parents and trashed=false"
            ).execute()
            files.extend(results.get('files', []))
            page_token = results.get('nextPageToken')
            if not page_token:
                break
        return files
    except Exception as e:
        with output:
            print(f"Error retrieving file list: {str(e)}")
        return []

def collect_folder_files(folder_id, folder_path="", service=None):
    if service is None:
        service = get_service()
    files_to_download = []
    items = list_files(folder_id)
    for item in items:
        item_path = os.path.join(folder_path, item['name'])
        if item['mimeType'] == 'application/vnd.google-apps.folder':
            sub_files = collect_folder_files(item['id'], item_path, service)
            files_to_download.extend(sub_files)
        else:
            files_to_download.append((item['id'], item_path))
    return files_to_download

def create_zip_from_folder(folder_id, folder_name, download_id):
    global is_downloading, current_download_id
    is_downloading = True
    current_download_id = download_id
    download_button.disabled = True
    progress_bar.layout.visibility = 'visible'
    progress_bar.value = 0.0
    zip_filename = None

    try:
        service = get_service()
        files_to_download = collect_folder_files(folder_id)
        if not files_to_download:
            with output:
                print("Folder is empty or contains no downloadable files.")
            return None

        total_files = len(files_to_download)
        zip_buffer = io.BytesIO()
        with zipfile.ZipFile(zip_buffer, 'w', zipfile.ZIP_DEFLATED) as zip_file:
            for i, (file_id, file_path) in enumerate(files_to_download):
                request = service.files().get_media(fileId=file_id)
                file_buffer = io.BytesIO()
                downloader = MediaIoBaseDownload(file_buffer, request)
                done = False
                while not done:
                    status, done = downloader.next_chunk()
                zip_file.writestr(file_path, file_buffer.getvalue())
                progress_bar.value = (i + 1) / total_files

        zip_buffer.seek(0)
        zip_filename = f"{folder_name}.zip"
        with open(zip_filename, 'wb') as f:
            f.write(zip_buffer.getvalue())

        time.sleep(10)  # Delay for stability
        files.download(zip_filename)
        with output:
            print(f"Archive {zip_filename} downloaded automatically.")

    except Exception as e:
        with output:
            print(f"Error creating archive: {str(e)}")

    finally:
        is_downloading = False
        current_download_id = None
        download_button.disabled = False
        progress_bar.layout.visibility = 'hidden'
        progress_bar.value = 0.0
    return zip_filename

def update_file_list():
    global current_folder_id
    items = list_files(current_folder_id)
    options = []
    for item in items:
        if item['mimeType'] == 'application/vnd.google-apps.folder':
            options.append((f"📁 {item['name']}", item['id']))
        else:
            options.append((f"📄 {item['name']}", item['id']))
    file_list.options = options

def on_refresh_click(b):
    with output:
        clear_output()
        update_file_list()

def on_go_click(b):
    global current_folder_id, folder_stack
    selected_id = file_list.value
    if selected_id:
        service = get_service()
        file_info = service.files().get(fileId=selected_id, fields='mimeType').execute()
        if file_info['mimeType'] == 'application/vnd.google-apps.folder':
            folder_stack.append(current_folder_id)
            current_folder_id = selected_id
            update_file_list()
            path_label.value = f"Current path: /{get_path()}"

def on_up_click(b):
    global current_folder_id, folder_stack
    if folder_stack:
        current_folder_id = folder_stack.pop()
        update_file_list()
        path_label.value = f"Current path: /{get_path()}"

def on_delete_click(b):
    selected_id = file_list.value
    if not selected_id:
        with output:
            clear_output()
            print("Select a file or folder to delete.")
        return
    service = get_service()
    file_info = service.files().get(fileId=selected_id, fields='name, mimeType').execute()
    item_name = file_info['name']
    service.files().delete(fileId=selected_id).execute()
    with output:
        clear_output()
        print(f"Deleted: {item_name}")
    update_file_list()

def on_download_click(b):
    global is_downloading, last_click_time, current_download_id
    current_time = time.time()
    if current_time - last_click_time < 3:
        with output:
            print("Please wait before clicking again...")
        return
    last_click_time = current_time

    if is_downloading:
        with output:
            print("Download already in progress, please wait.")
        return

    selected_id = file_list.value
    if not selected_id:
        with output:
            print("Select a file or folder to download.")
        return

    service = get_service()
    file_info = service.files().get(fileId=selected_id, fields='mimeType, name').execute()
    download_id = str(uuid.uuid4())
    with output:
        clear_output()
    if file_info['mimeType'] == 'application/vnd.google-apps.folder':
        create_zip_from_folder(selected_id, file_info['name'], download_id)
    else:
        request = service.files().get_media(fileId=selected_id)
        fh = io.BytesIO()
        downloader = MediaIoBaseDownload(fh, request)
        done = False
        while not done:
            status, done = downloader.next_chunk()
        with open(file_info['name'], 'wb') as f:
            f.write(fh.getvalue())
        time.sleep(10)
        files.download(file_info['name'])
        with output:
            print(f"File {file_info['name']} downloaded automatically.")

def on_upload_click(b):
    with output:
        clear_output()
        print("Select files to upload...")
    try:
        uploaded = files.upload()  # Запрашиваем файлы у пользователя
        if not uploaded:
            with output:
                print("No files selected.")
            return
        service = get_service()
        for file_name, file_content in uploaded.items():
            # Создаем временный файл
            with tempfile.NamedTemporaryFile(delete=False, suffix=os.path.splitext(file_name)[1]) as temp_file:
                temp_file.write(file_content)
                temp_file_path = temp_file.name

            file_metadata = {
                'name': file_name,
                'parents': [current_folder_id]
            }
            media = MediaFileUpload(
                temp_file_path,
                mimetype='application/octet-stream',
                resumable=True
            )
            service.files().create(
                body=file_metadata,
                media_body=media,
                fields='id'
            ).execute()
            # Удаляем временный файл
            os.unlink(temp_file_path)
            with output:
                print(f"Uploaded: {file_name}")
        update_file_list()  # Обновляем список файлов после загрузки
    except Exception as e:
        with output:
            print(f"Error uploading files: {str(e)}")

def get_path():
    path = []
    temp_folder_id = current_folder_id
    service = get_service()
    while temp_folder_id != 'root':
        file_info = service.files().get(fileId=temp_folder_id, fields='name, parents').execute()
        path.insert(0, file_info['name'])
        parents = file_info.get('parents', [])
        if not parents:
            break
        temp_folder_id = parents[0]
    return '/'.join(path) if path else ''

refresh_button.on_click(on_refresh_click)
go_button.on_click(on_go_click)
up_button.on_click(on_up_click)
download_button.on_click(on_download_click)
delete_button.on_click(on_delete_click)
upload_button.on_click(on_upload_click)

display(json_selector)
display(refresh_button)
display(path_label)
display(file_list)
display(go_button)
display(up_button)
display(download_button)
display(delete_button)
display(upload_button)
display(progress_bar)
display(output)

update_file_list()

In [ ]:
#@title ##**Test folders** { display-mode: "form" }
from googleapiclient.http import MediaFileUpload

# Function to create a test folder and file, and share the folder
def create_and_share_test_folder():
    try:
        # Connect to Google Drive
        SCOPES = ['https://www.googleapis.com/auth/drive']
        credentials = Credentials.from_service_account_file(
            os.path.join('service_accounts', json_selector.value),
            scopes=SCOPES
        )
        service = build('drive', 'v3', credentials=credentials)

        # Extract user email from the JSON file name
        json_file_name = json_selector.value.split('.')[0]
        user_email = f"{json_file_name}@gmail.com"

        # 1. Create test folder
        folder_metadata = {
            'name': 'Test Folder',
            'mimeType': 'application/vnd.google-apps.folder'
        }
        folder = service.files().create(body=folder_metadata, fields='id').execute()
        folder_id = folder.get('id')
        print(f"Test folder created with ID: {folder_id}")

        # 2. Create test file in the folder
        file_metadata = {
            'name': 'test_file.txt',
            'parents': [folder_id]
        }
        with open('test_file.txt', 'w') as f:
            f.write('This is a test file.')

        media = MediaFileUpload('test_file.txt', mimetype='text/plain')
        file = service.files().create(body=file_metadata, media_body=media, fields='id').execute()
        print(f"Test file created with ID: {file.get('id')}")

        # Remove local file after upload
        os.remove('test_file.txt')

        # 3. Share the folder with the user
        permission = {
            'type': 'user',
            'role': 'writer',
            'emailAddress': user_email
        }
        service.permissions().create(
            fileId=folder_id,
            body=permission,
            fields='id'
        ).execute()
        print(f"Folder shared with user: {user_email}")

    except Exception as e:
        print(f"An error occurred: {str(e)}")

# Run the function
create_and_share_test_folder()
